In [1]:
# Import basic modules, libraries, and arguments
from parsing import *

# Create "figure" directory
if not os.path.exists(args.figure_directory):
    print('Figure directory does not exist: Created Figure directory\n')
    os.mkdir(args.figure_directory)

# Read permeability map
perm_field = []

with open(os.path.join(args.basicfilepath, args.perm_filename + ".DATA"), 'r') as file_read:
    lines = file_read.readlines()
    for line_num in range(args.gridnum_x * args.gridnum_y):
        perm_field.append(float(lines[line_num + 1]))

### 1. Well placement map from simulation file

In [4]:
####################################################################################
######################## Adjust variables in here if needed ########################

args.step_progress = 20 # When simulation sampling steps were stopped, Adjust here.

####################################################################################
####################################################################################

# (1) Gather well placement data from simulation file
well_placement_dict = {}

for step_num in range(1, args.step_progress+1):
    if not os.path.exists(os.path.join(args.figure_directory, f'Step{step_num}-WP')):
        os.mkdir(os.path.join(args.figure_directory, f'Step{step_num}-WP'))

    if not os.path.exists(os.path.join(args.figure_directory, f'Step{step_num}-WP', 'WellPlacement')):
        os.mkdir(os.path.join(args.figure_directory, f'Step{step_num}-WP', 'WellPlacement'))

    for sample_num in range(1, args.sample_num_per_iter+1):
        well_loc_list = []
        # Read Well placement of each sample from simulation file
        with open(os.path.join(args.simulation_directory, f"Step{step_num}_Sample{sample_num}", f"{args.well_filename}_Sam{sample_num}_Seq{args.total_well_num_max}.DATA")) as file_read:
            line = file_read.readline()
            for tstep_idx in range(0, args.total_well_num_max):
                line = file_read.readline()
                line = file_read.readline()
                line_list = [element.strip() for element in line.split()]
                well_loc = (int(line_list[2]), int(line_list[3]))
                well_loc_list.append(well_loc)
                if tstep_idx == args.total_well_num_max - 1:
                    break
                while not line.startswith("--WELL"):
                    line = file_read.readline()
        # Save Well placement at dictionary [Key: Step{}_Sample{}, Value: list[well_loc -> tuple]]
        well_placement_dict[f'Step{step_num}_Sample{sample_num}'] = well_loc_list

In [ ]:
# (2) Draw and Save all well location map at each Step [Total num. of figures: args.step_progress]
for step_num in range(1, args.step_progress+1):
    list_x = []
    list_y = []

    for sample_num in range(1, args.sample_num_per_iter+1):
        for well_location in range(len(well_placement_dict[f'Step{step_num}_Sample{sample_num}'])):
            list_x.append(well_placement_dict[f'Step{step_num}_Sample{sample_num}'][well_location][0]-1) # Grid index: 0~(nx-1)
            list_y.append(well_placement_dict[f'Step{step_num}_Sample{sample_num}'][well_location][1]-1) # Grid index: 0~(ny-1)
    plt.figure(figsize=(8,6)) # Unit: inch
    plt.tight_layout()
    plt.scatter(list_x, list_y, c='k')
    plt.imshow(np.log(np.array(perm_field)).reshape(15,15), cmap='jet')
    cbar = plt.colorbar()
    cbar.ax.set_ylabel("ln(Perm)")
    plt.gca().xaxis.tick_top()
    plt.gca().set_xticks(range(0, 15))
    plt.gca().set_xlabel("Grid X", loc='center')
    plt.gca().xaxis.set_label_position('top')
    plt.gca().set_yticks(range(0, 15))
    plt.gca().set_ylabel("Grid Y")
    plt.gca().set_title(f"ln(Perm) map with all well locations (Step #{step_num}, Total Sample Num. {args.sample_num_per_iter})", font="Arial", fontsize=16)
    figname = f'All Well placement-Step{step_num}' + '.png'
    plt.savefig(os.path.join(args.figure_directory, figname))
    plt.close()

In [13]:
# (3) Draw and Save well placement map for each Sample [Total num. of figures: args.sample_num_per_iter * args.step_progress]
for step_num in range(1, args.step_progress+1):
    for sample_num in range(1, args.sample_num_per_iter+1):
        list_x = []
        list_y = []
        list_num = [i for i in range(1, args.total_well_num_max+1)]
        for well_location in range(len(well_placement_dict[f'Step{step_num}_Sample{sample_num}'])):
            list_x.append(well_placement_dict[f'Step{step_num}_Sample{sample_num}'][well_location][0]-1) # Grid index: 0~(nx-1)
            list_y.append(well_placement_dict[f'Step{step_num}_Sample{sample_num}'][well_location][1]-1) # Grid index: 0~(ny-1)
        plt.figure(figsize=(8,6)) # Unit: inch
        plt.tight_layout()
        plt.scatter(list_x, list_y, c='k')
        for i, txt in enumerate(list_num):
            plt.gca().text(list_x[i]+0.3, list_y[i]+0.3, txt, fontsize=10)
        plt.imshow(np.log(np.array(perm_field)).reshape(15,15), cmap='jet')
        cbar = plt.colorbar()
        cbar.ax.set_ylabel("ln(Perm)")
        plt.gca().xaxis.tick_top()
        plt.gca().set_xticks(range(0, 15))
        plt.gca().set_xlabel("Grid X", loc='center')
        plt.gca().xaxis.set_label_position('top')
        plt.gca().set_yticks(range(0, 15))
        plt.gca().set_ylabel("Grid Y")
        plt.gca().set_title(f"ln(Perm) map with well location (Step #{step_num}, Sample #{sample_num})", font="Arial", fontsize=16)
        figname = f'Well placement-Step{step_num}-Sample{sample_num}' + '.png'
        plt.savefig(os.path.join(args.figure_directory, f'Step{step_num}-WP', 'WellPlacement', figname))
        plt.close()

In [5]:
# (4) Draw and Save the location of 1st well at each Step [Exclude last Exploitation step] [Total num. of figures: args.step_progress]
for step_num in range(1, args.step_progress+1):
    list_x = []
    list_y = []
    for sample_num in range(1, args.sample_num_per_iter+1):
        # 1st well location: well_placement_dict[f'Step{step_num}_Sample{sample_num}'][0]
        list_x.append(well_placement_dict[f'Step{step_num}_Sample{sample_num}'][0][0]-1) # Grid index: 0 ~ (nx-1)
        list_y.append(well_placement_dict[f'Step{step_num}_Sample{sample_num}'][0][1]-1) # Grid index: 0 ~ (ny-1)
    plt.figure(figsize=(8,6)) # Unit: inch
    plt.tight_layout()
    plt.scatter(list_x, list_y, c='k')
    plt.imshow(np.log(np.array(perm_field)).reshape(15,15), cmap='jet')
    cbar = plt.colorbar()
    cbar.ax.set_ylabel("ln(Perm)")
    plt.gca().xaxis.tick_top()
    plt.gca().set_xticks(range(0, 15))
    plt.gca().set_xlabel("Grid X", loc='center')
    plt.gca().xaxis.set_label_position('top')
    plt.gca().set_yticks(range(0, 15))
    plt.gca().set_ylabel("Grid Y")
    plt.gca().set_title(f"ln(Perm) map with 1st well locations (Step #{step_num}, Total Sample Num. {args.sample_num_per_iter})", font="Arial", fontsize=16)
    figname = f'1st Well placement-Step{step_num}' + '.png'
    plt.savefig(os.path.join(args.figure_directory, figname))
    plt.close()

In [6]:
# (5) Draw and Save the location of 2nd well at each Step [Exclude last Exploitation step] [Total num. of figures: args.step_progress]
for step_num in range(1, args.step_progress+1):
    list_x = []
    list_y = []
    for sample_num in range(1, args.sample_num_per_iter+1):
        # 1st well location: well_placement_dict[f'Step{step_num}_Sample{sample_num}'][1]
        list_x.append(well_placement_dict[f'Step{step_num}_Sample{sample_num}'][1][0]-1) # Grid index: 0 ~ (nx-1)
        list_y.append(well_placement_dict[f'Step{step_num}_Sample{sample_num}'][1][1]-1) # Grid index: 0 ~ (ny-1)
    plt.figure(figsize=(8,6)) # Unit: inch
    plt.tight_layout()
    plt.scatter(list_x, list_y, c='k')
    plt.imshow(np.log(np.array(perm_field)).reshape(15,15), cmap='jet')
    cbar = plt.colorbar()
    cbar.ax.set_ylabel("ln(Perm)")
    plt.gca().xaxis.tick_top()
    plt.gca().set_xticks(range(0, 15))
    plt.gca().set_xlabel("Grid X", loc='center')
    plt.gca().xaxis.set_label_position('top')
    plt.gca().set_yticks(range(0, 15))
    plt.gca().set_ylabel("Grid Y")
    plt.gca().set_title(f"ln(Perm) map with 2nd well locations (Step #{step_num}, Total Sample Num. {args.sample_num_per_iter})", font="Arial", fontsize=16)
    figname = f'2nd Well placement-Step{step_num}' + '.png'
    plt.savefig(os.path.join(args.figure_directory, figname))
    plt.close()

### 2. NPV from simulation file

In [7]:
####################################################################################
######################## Adjust variables in here if needed ########################

args.step_progress = 20 # When simulation sampling steps were stopped, Adjust here.

####################################################################################
####################################################################################

# (1) Gather NPV data from simulation file and calculate Average NPV
production_dict_FOPT = {}
production_dict_FWPT = {}
production_dict_FWIT = {}

for step_num in range(1, args.step_progress+1):
    if not os.path.exists(os.path.join(args.figure_directory, f'Step{step_num}-WP', 'NPV')):
        os.mkdir(os.path.join(args.figure_directory, f'Step{step_num}-WP', 'NPV'))
    for sample_num in range(1, args.sample_num_per_iter+1):
        FOPT_list = []
        FWPT_list = []
        FWIT_list = []
        # Read production data of each sample
        with open(os.path.join(args.simulation_directory, f"Step{step_num}_Sample{sample_num}", f"{args.ecl_filename}_SAM{sample_num}_SEQ{args.total_well_num_max}.RSM")) as file_read:
            for i in range(10):
                line = file_read.readline()
            for tstep_idx in range(0, args.total_well_num_max+1):
                line_list = [element.strip() for element in line.split()]
                FOPT_list.append(float(line_list[2]))
                FWPT_list.append(float(line_list[3]))
                FWIT_list.append(float(line_list[4]))
                line = file_read.readline()
        # Save Well placement at dictionary [Key: Step{}_Sample{}, Value: list[Prod. data -> float]]
        production_dict_FOPT[f'Step{step_num}_Sample{sample_num}'] = FOPT_list
        production_dict_FWPT[f'Step{step_num}_Sample{sample_num}'] = FWPT_list
        production_dict_FWIT[f'Step{step_num}_Sample{sample_num}'] = FWIT_list

npv_dict = {}

for step_num in range(1, args.step_progress+1):
    for sample_num in range(1, args.sample_num_per_iter+1):
        income_list = []
        for tstep_idx in range(0, args.total_well_num_max):
            oil_income = (production_dict_FOPT[f'Step{step_num}_Sample{sample_num}'][tstep_idx+1] - production_dict_FOPT[f'Step{step_num}_Sample{sample_num}'][tstep_idx]) * args.oil_price / (((1 + args.discount_rate)) ** (args.time_step * (tstep_idx + 1) / 365))
            water_treat = (production_dict_FWPT[f'Step{step_num}_Sample{sample_num}'][tstep_idx+1] - production_dict_FWPT[f'Step{step_num}_Sample{sample_num}'][tstep_idx]) * args.water_treatment / (((1 + args.discount_rate)) ** (args.time_step * (tstep_idx + 1) / 365))
            water_inj = (production_dict_FWIT[f'Step{step_num}_Sample{sample_num}'][tstep_idx+1] - production_dict_FWIT[f'Step{step_num}_Sample{sample_num}'][tstep_idx]) * args.water_injection / (((1 + args.discount_rate)) ** (args.time_step * (tstep_idx + 1) / 365))
            income_list.append(oil_income - water_treat - water_inj)
        npv_dict[f'Step{step_num}_Sample{sample_num}'] = list(np.cumsum(np.array(income_list)))


avg_npv_dict = {}

for step_num in range(1, args.step_progress+1):
    time_step_list = [args.time_step * i for i in range(args.total_well_num_max + 1)]
    npv_array = np.zeros(len(time_step_list))
    for sample_num in range(1, args.sample_num_per_iter + 1):
        for tstep_idx in range(args.total_well_num_max): # Initial NPV == 0 for this case
            npv_array[tstep_idx+1] += npv_dict[f'Step{step_num}_Sample{sample_num}'][tstep_idx]
    avg_npv_dict[f'Step{step_num}'] = list(npv_array / args.sample_num_per_iter)

In [6]:
# (2) Draw and Save NPV graph for each Sample [Total num. of figures: args.sample_num_per_iter * args.step_progress]
for step_num in range(1, args.step_progress+1):
    for sample_num in range(1, args.sample_num_per_iter+1):
        time_step_list = [args.time_step * i for i in range(args.total_well_num_max+1)]
        plt.figure(figsize=(8,6)) # Unit: inch
        plt.plot(time_step_list, np.array([0] + npv_dict[f'Step{step_num}_Sample{sample_num}'])/(10**6))
        plt.gca().set_xlim(time_step_list[0], time_step_list[-1])
        plt.xticks(time_step_list)
        plt.gca().set_xlabel("Days", loc='center')
        plt.gca().set_ylim(0, 18)
        plt.gca().set_ylabel("NPV, MM$", loc='center')
        plt.grid()
        plt.gca().set_title(f"NPV (MM$, Step #{step_num}, Sample #{sample_num})", font="Arial", fontsize=16)
        figname = f'NPV-Step{step_num}-Sample{sample_num}' + '.png'
        plt.savefig(os.path.join(args.figure_directory, f'Step{step_num}-WP', 'NPV', figname))
        plt.close()

In [8]:
# (3) Draw and Save Average NPV graph with all samples in each step [Total num. of figures: args.step_progress]
for step_num in range(1, args.step_progress+1):
    time_step_list = [args.time_step * i for i in range(args.total_well_num_max + 1)]

    # Graph of NPV of all samples
    plt.figure(figsize=(8, 6))  # Unit: inch
    for sample_num in range(1, args.sample_num_per_iter+1): # Initial NPV == 0 for this case
        plt.plot(time_step_list, np.array([0] + npv_dict[f'Step{step_num}_Sample{sample_num}']) / (10 ** 6), color='silver')
    plt.gca().set_xlim(time_step_list[0], time_step_list[-1])
    plt.xticks(time_step_list)
    plt.gca().set_xlabel("Days", loc='center')
    plt.gca().set_ylim(0, 18)
    plt.gca().set_ylabel("NPV, MM$", loc='center')
    plt.grid()
    plt.gca().set_title(f"NPV Value (MM$, Step #{step_num})", font="Arial", fontsize=16)

    # Graph of Average NPV of all samples
    plt.plot(time_step_list, np.array(avg_npv_dict[f'Step{step_num}']) / (10 ** 6), color='orange') # Initial NPV == 0 for this case
    plt.gca().legend(['NPV of each samples', 'Average NPV for all samples'], loc='upper left')

    ax = plt.gca()
    leg = ax.get_legend()
    leg.legend_handles[0].set_color('silver') # Legend: 'NPV of each samples'
    leg.legend_handles[1].set_color('orange') # Legend: 'Average NPV for all samples'

    figname = f'NPV-Step{step_num}' + '.png'
    plt.savefig(os.path.join(args.figure_directory, figname))
    plt.close()

In [9]:
# (4) NPV variation at each sampling step [Exclude last Exploitation step] [Total num. of figures: 1]
sampling_step = [i for i in range(args.step_progress+1)]
avg_npv_for_step = [0]

for step_num in range(1, args.step_progress+1):
    avg_npv_for_step.append(avg_npv_dict[f'Step{step_num}'][-1]) # Average NPV at last time

# Graph of Average NPV at each step
plt.figure(figsize=(8, 6))  # Unit: inch
plt.plot(sampling_step, np.array(avg_npv_for_step) / (10 ** 6), color='red')
plt.gca().set_xlim(sampling_step[0], sampling_step[-1])
plt.xticks(sampling_step)
plt.gca().set_xlabel("Steps", loc='center')
plt.gca().set_ylim(0, 16)
plt.gca().set_ylabel("Average NPV, MM$", loc='center')
plt.grid()
plt.gca().set_title(f"Average NPV Value Variation by Sampling Steps", font="Arial", fontsize=16)

figname = f'Average NPV variation' + '.png'
plt.savefig(os.path.join(args.figure_directory, figname))
plt.close()

### Figure for last exploitation results

In [10]:
# # # For last exploitation results (Simulation files)
#
####################################################################################
######################## Adjust variables in here if needed ########################

args.step_progress = args.max_iteration + 1
args.sample_num_per_iter = 10 # Last sampling for Exploitation

####################################################################################
####################################################################################
#
# well_placement_dict = {}
#
# for step_num in range(args.step_progress, args.step_progress+1): # When simulation sampling steps were stopped
#     if not os.path.exists(os.path.join(args.figure_directory, f'Step{step_num}-WP')):
#         os.mkdir(os.path.join(args.figure_directory, f'Step{step_num}-WP'))
#
#     if not os.path.exists(os.path.join(args.figure_directory, f'Step{step_num}-WP', 'WellPlacement')):
#         os.mkdir(os.path.join(args.figure_directory, f'Step{step_num}-WP', 'WellPlacement'))
#
#     for sample_num in range(1, args.sample_num_per_iter+1):
#         well_loc_list = []
#         # Read Well placement of each sample from simulation file
#         with open(os.path.join(args.simulation_directory, f"Step{step_num}_Sample{sample_num}", f"{args.well_filename}_Sam{sample_num}_Seq{args.total_well_num_max}.DATA")) as file_read:
#             line = file_read.readline()
#             for tstep_idx in range(0, args.total_well_num_max):
#                 line = file_read.readline()
#                 line = file_read.readline()
#                 line_list = [element.strip() for element in line.split()]
#                 well_loc = (int(line_list[2]), int(line_list[3]))
#                 well_loc_list.append(well_loc)
#                 if tstep_idx == args.total_well_num_max - 1:
#                     break
#                 while not line.startswith("--WELL"):
#                     line = file_read.readline()
#         # Save Well placement at dictionary [Key: Step{}_Sample{}, Value: list[well_loc -> tuple]]
#         well_placement_dict[f'Step{step_num}_Sample{sample_num}'] = well_loc_list
#
# # Draw all well location map of samples at each Step
# for step_num in range(args.step_progress, args.step_progress+1):
#     list_x = []
#     list_y = []
#
#     for sample_num in range(1, args.sample_num_per_iter+1):
#         for well_location in range(len(well_placement_dict[f'Step{step_num}_Sample{sample_num}'])):
#             list_x.append(well_placement_dict[f'Step{step_num}_Sample{sample_num}'][well_location][0]-1) # Grid index: 0~(nx-1)
#             list_y.append(well_placement_dict[f'Step{step_num}_Sample{sample_num}'][well_location][1]-1) # Grid index: 0~(ny-1)
#     plt.figure(figsize=(8,6)) # Unit: inch
#     plt.tight_layout()
#     plt.scatter(list_x, list_y, c='k')
#     plt.imshow(np.log(np.array(perm_field)).reshape(15,15), cmap='jet')
#     cbar = plt.colorbar()
#     cbar.ax.set_ylabel("ln(Perm)")
#     plt.gca().xaxis.tick_top()
#     plt.gca().set_xticks(range(0, 15))
#     plt.gca().set_xlabel("Grid X", loc='center')
#     plt.gca().xaxis.set_label_position('top')
#     plt.gca().set_yticks(range(0, 15))
#     plt.gca().set_ylabel("Grid Y")
#     plt.gca().set_title(f"ln(Perm) map with all well locations (Step #{step_num}, Total Sample Num. {args.sample_num_per_iter})", font="Arial", fontsize=16)
#     figname = f'All Well placement-Step{step_num}' + '.png'
#     plt.savefig(os.path.join(args.figure_directory, figname))
#     plt.close()
#
# # Draw well placement map for each sample
# for step_num in range(args.step_progress, args.step_progress+1):
#     for sample_num in range(1, args.sample_num_per_iter+1):
#         list_x = []
#         list_y = []
#         list_num = [i for i in range(1, args.total_well_num_max+1)]
#         for well_location in range(len(well_placement_dict[f'Step{step_num}_Sample{sample_num}'])):
#             list_x.append(well_placement_dict[f'Step{step_num}_Sample{sample_num}'][well_location][0]-1) # Grid index: 0~(nx-1)
#             list_y.append(well_placement_dict[f'Step{step_num}_Sample{sample_num}'][well_location][1]-1) # Grid index: 0~(ny-1)
#         plt.figure(figsize=(8,6)) # Unit: inch
#         plt.tight_layout()
#         plt.scatter(list_x, list_y, c='k')
#         for i, txt in enumerate(list_num):
#             plt.gca().text(list_x[i]+0.3, list_y[i]+0.3, txt, fontsize=10)
#         plt.imshow(np.log(np.array(perm_field)).reshape(15,15), cmap='jet')
#         cbar = plt.colorbar()
#         cbar.ax.set_ylabel("ln(Perm)")
#         plt.gca().xaxis.tick_top()
#         plt.gca().set_xticks(range(0, 15))
#         plt.gca().set_xlabel("Grid X", loc='center')
#         plt.gca().xaxis.set_label_position('top')
#         plt.gca().set_yticks(range(0, 15))
#         plt.gca().set_ylabel("Grid Y")
#         plt.gca().set_title(f"ln(Perm) map with well location (Step #{step_num}, Sample #{sample_num})", font="Arial", fontsize=16)
#         figname = f'Well placement-Step{step_num}-Sample{sample_num}' + '.png'
#         plt.savefig(os.path.join(args.figure_directory, f'Step{step_num}-WP', 'WellPlacement', figname))
#         plt.close()

# Gather NPV data from simulation file
production_dict_FOPT_last = {}
production_dict_FWPT_last = {}
production_dict_FWIT_last = {}

for step_num in range(args.step_progress, args.step_progress+1):
    if not os.path.exists(os.path.join(args.figure_directory, f'Step{step_num}-WP', 'NPV')):
        os.mkdir(os.path.join(args.figure_directory, f'Step{step_num}-WP', 'NPV'))
    for sample_num in range(1, args.sample_num_per_iter+1):
        FOPT_list = []
        FWPT_list = []
        FWIT_list = []
        # Read production data of each sample
        with open(os.path.join(args.simulation_directory, f"Step{step_num}_Sample{sample_num}", f"{args.ecl_filename}_SAM{sample_num}_SEQ{args.total_well_num_max}.RSM")) as file_read:
            for i in range(10):
                line = file_read.readline()
            for tstep_idx in range(0, args.total_well_num_max+1):
                line_list = [element.strip() for element in line.split()]
                FOPT_list.append(float(line_list[2]))
                FWPT_list.append(float(line_list[3]))
                FWIT_list.append(float(line_list[4]))
                line = file_read.readline()
        # Save Well placement at dictionary [Key: Step{}_Sample{}, Value: list[Prod. data -> float]]
        production_dict_FOPT_last[f'Step{step_num}_Sample{sample_num}'] = FOPT_list
        production_dict_FWPT_last[f'Step{step_num}_Sample{sample_num}'] = FWPT_list
        production_dict_FWIT_last[f'Step{step_num}_Sample{sample_num}'] = FWIT_list

npv_dict_last = {}

for step_num in range(args.step_progress, args.step_progress+1):
    for sample_num in range(1, args.sample_num_per_iter+1):
        income_list = []
        for tstep_idx in range(0, args.total_well_num_max):
            oil_income = (production_dict_FOPT_last[f'Step{step_num}_Sample{sample_num}'][tstep_idx+1] - production_dict_FOPT_last[f'Step{step_num}_Sample{sample_num}'][tstep_idx]) * args.oil_price / (((1 + args.discount_rate)) ** (args.time_step * (tstep_idx + 1) / 365))
            water_treat = (production_dict_FWPT_last[f'Step{step_num}_Sample{sample_num}'][tstep_idx+1] - production_dict_FWPT_last[f'Step{step_num}_Sample{sample_num}'][tstep_idx]) * args.water_treatment / (((1 + args.discount_rate)) ** (args.time_step * (tstep_idx + 1) / 365))
            water_inj = (production_dict_FWIT_last[f'Step{step_num}_Sample{sample_num}'][tstep_idx+1] - production_dict_FWIT_last[f'Step{step_num}_Sample{sample_num}'][tstep_idx]) * args.water_injection / (((1 + args.discount_rate)) ** (args.time_step * (tstep_idx + 1) / 365))
            income_list.append(oil_income - water_treat - water_inj)
        npv_dict_last[f'Step{step_num}_Sample{sample_num}'] = list(np.cumsum(np.array(income_list)))

# # Draw NPV of each sample
# for step_num in range(args.step_progress, args.step_progress+1):
#     for sample_num in range(1, args.sample_num_per_iter+1):
#         time_step_list = [args.time_step * i for i in range(args.total_well_num_max+1)]
#         plt.figure(figsize=(8,6)) # Unit: inch
#         plt.plot(time_step_list, np.array([0] + npv_dict_last[f'Step{step_num}_Sample{sample_num}'])/(10**6))
#         plt.gca().set_xlim(time_step_list[0], time_step_list[-1])
#         plt.xticks(time_step_list)
#         plt.gca().set_xlabel("Days", loc='center')
#         plt.gca().set_ylim(0, 18)
#         plt.gca().set_ylabel("NPV, MM$", loc='center')
#         plt.grid()
#         plt.gca().set_title(f"NPV (MM$, Step #{step_num}, Sample #{sample_num})", font="Arial", fontsize=16)
#         figname = f'NPV-Step{step_num}-Sample{sample_num}' + '.png'
#         plt.savefig(os.path.join(args.figure_directory, f'Step{step_num}-WP', 'NPV', figname))
#         plt.close()

# Average NPV with all samples in each step
avg_npv_dict_last = {}

for step_num in range(args.step_progress, args.step_progress+1):
    time_step_list = [args.time_step * i for i in range(args.total_well_num_max + 1)]

    # # Graph of NPV of all samples
    # plt.figure(figsize=(8, 6))  # Unit: inch
    # for sample_num in range(1, args.sample_num_per_iter+1): # Initial NPV == 0 for this case
    #     plt.plot(time_step_list, np.array([0] + npv_dict_last[f'Step{step_num}_Sample{sample_num}']) / (10 ** 6), color='silver')
    # plt.gca().set_xlim(time_step_list[0], time_step_list[-1])
    # plt.xticks(time_step_list)
    # plt.gca().set_xlabel("Days", loc='center')
    # plt.gca().set_ylim(0, 18)
    # plt.gca().set_ylabel("NPV, MM$", loc='center')
    # plt.grid()
    # plt.gca().set_title(f"NPV Value (MM$, Step {step_num})", font="Arial", fontsize=16)

    # Graph of Average NPV for all samples
    npv_array = np.zeros(len(time_step_list))
    for sample_num in range(1, args.sample_num_per_iter + 1):
        for tstep_idx in range(args.total_well_num_max): # Initial NPV == 0 for this case
            npv_array[tstep_idx+1] += npv_dict_last[f'Step{step_num}_Sample{sample_num}'][tstep_idx]
    avg_npv_dict_last[f'Step{step_num}'] = list(npv_array / args.sample_num_per_iter)
    # plt.plot(time_step_list, np.array(avg_npv_dict_last[f'Step{step_num}']) / (10 ** 6), color='orange') # Initial NPV == 0 for this case
    # plt.gca().legend(['NPV of each samples', 'Average NPV for all samples'], loc='upper left')
    #
    # ax = plt.gca()
    # leg = ax.get_legend()
    # leg.legend_handles[0].set_color('silver') # Legend: 'NPV of each samples'
    # leg.legend_handles[1].set_color('orange') # Legend: 'Average NPV for all samples'
    #
    # figname = f'NPV-Step{step_num}' + '.png'
    # plt.savefig(os.path.join(args.figure_directory, figname))
    # plt.close()

In [11]:
# NPV variation at each sampling step [Include last Exploitation step] [Total num. of figures: 1]
sampling_step = [i for i in range(args.step_progress+1)]
avg_npv_for_step = [0]

for step_num in range(1, args.step_progress+1):
    if step_num < args.step_progress:
        avg_npv_for_step.append(avg_npv_dict[f'Step{step_num}'][-1]) # Average NPV at last time
    else:
        avg_npv_for_step.append(avg_npv_dict_last[f'Step{step_num}'][-1])

# Graph of Average NPV at each step
plt.figure(figsize=(8, 6))  # Unit: inch
plt.plot(sampling_step, np.array(avg_npv_for_step) / (10 ** 6), color='red')
plt.gca().set_xlim(sampling_step[0], sampling_step[-1])
plt.xticks(sampling_step)
plt.gca().set_xlabel("Steps", loc='center')
plt.gca().set_ylim(0, 16)
plt.gca().set_ylabel("Average NPV, MM$", loc='center')
plt.grid()
plt.gca().set_title(f"Average NPV Value Variation by Sampling Steps", font="Arial", fontsize=16)

figname = f'Average NPV variation with last exploitation' + '.png'
plt.savefig(os.path.join(args.figure_directory, figname))
plt.close()

### 3. Well placement map from .pkl

In [26]:
# from parsing import *
#
# ################################### Class: Placement sample #################################### Certified
# # WellPlacementSample Class contains information of One full sequence of simulation sample.
# class WellPlacementSample:
#     def __init__(self, args):
#         self.args = args
#         self.well_loc_map = [[[0 for i in range(0, args.gridnum_x)] for j in range(0, args.gridnum_y)]]
#         self.well_loc_list = []
#         self.PRESSURE_map = [[[args.initial_PRESSURE for i in range(0, args.gridnum_x)] for j in range(0, args.gridnum_y)]]
#         self.SOIL_map = [[[args.initial_SOIL for i in range(0, args.gridnum_x)] for j in range(0, args.gridnum_y)]]
#         self.income = []
#
# ################################### Class: Experience sample ################################### Certified
# # Experience Class contains One set of (s, a, r, s').
# class Experience:
#     def __init__(self, args):
#         self.args = args
#         self.current_state = list
#         self.current_action = None
#         self.reward = None
#         self.next_state = list
#
#     def __transform__(self):
#         self.current_state = torch.tensor(data=self.current_state, dtype=torch.float, device=self.args.device, requires_grad=False)
#         self.current_action = torch.tensor(data=self.current_action, dtype=torch.float, device=self.args.device, requires_grad=False)
#         self.reward = torch.tensor(data=self.reward, dtype=torch.float, device=self.args.device, requires_grad=False)
#         self.next_state = torch.tensor(data=self.next_state, dtype=torch.float, device=self.args.device, requires_grad=False)
#
#     def transform(self):
#         self.__transform__()
#
#
# ###################### Class: Experience sample (Dataset for DataLoader) ####################### Certified
# class Experience_list(Dataset):
#     def __init__(self, args):
#         self.args = args
#         self.exp_list = deque()
#
#     def __len__(self):
#         return len(self.exp_list)
#
#     def __getitem__(self, idx):
#         return self.exp_list[idx].current_state
#
# with open('variables\\Simulation_sample\\Simulation_sample_1.pkl', 'rb') as simsam:
#     Simulation_sample = pickle.load(simsam)
#
# npv_dict = {}
# args.reward_unit = 10 ** 8 # For scaling
#
# # for step_num in range(1, args.max_iteration+1):
# for sample_num in range(1, args.sample_num_per_iter+1):
#     npv_dict[f'Step{1}_Sample{sample_num}'] = [0] + list(np.cumsum(np.array(Simulation_sample[sample_num-1].income)) * args.reward_unit)
#
# avg_npv_dict = {}

In [39]:

# time_step_list = [args.time_step * i for i in range(args.total_well_num_max+1)]
# plt.figure(figsize=(8,6)) # Unit: inch
# for sample_num in range(1, args.sample_num_per_iter+1):
#     plt.plot(time_step_list, np.array(npv_dict[f'Step{1}_Sample{sample_num}'])/(10**6), color='silver')
# plt.gca().set_xlim(time_step_list[0], time_step_list[-1])
# plt.xticks(time_step_list)
# plt.gca().set_xlabel("Days", loc='center')
# plt.gca().set_ylim(0, 18)
# plt.gca().set_ylabel("NPV, MM$", loc='center')
# plt.grid()
# plt.gca().set_title(f"NPV Value (MM$, Step {1})", font="Arial", fontsize=16)
#
# npv_array = np.zeros(len(time_step_list))
# for sample_num in range(1, args.sample_num_per_iter+1):
#     for tstep_idx in range(args.total_well_num_max):
#         npv_array[tstep_idx+1] += npv_dict[f'Step{1}_Sample{sample_num}'][tstep_idx+1]
# avg_npv_dict[f'Step{1}'] = list(npv_array / args.sample_num_per_iter)
# plt.plot(time_step_list, np.array(avg_npv_dict[f'Step{1}'])/(10**6), color='orange')
# plt.gca().legend(['NPV of each samples', 'Average NPV for all samples'], loc='upper left')
#
# ax = plt.gca()
# leg = ax.get_legend()
# leg.legend_handles[0].set_color('silver')
# leg.legend_handles[1].set_color('orange')
#
# figname = f'NPV-Step{1}' + '.png'
# plt.savefig(os.path.join('figure', figname))
# plt.close()